In [1]:
import pandas as pd
import numpy as np
import math
from scipy import spatial

In [49]:
kp = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KenPom Team Metrics.csv')
#kp.head(10)
kp_conf_rat = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KP_Conf_Ratings.csv')
kp_conf_rat = kp_conf_rat[['Year_Conf', 'Conf_Abb', 'Conf_Rk', 'Conf_Full', 'Conf_Rating']]

d = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv')
d = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = df.replace(to_replace='NO IDEA', value=0)
df = df.replace(to_replace='---', value=0)
df = df.fillna(0)
df = df.drop(df.loc[((df['games']>39) & (df['year']==2023)) | ((df['games_n1']>39) & (df['year_n1']==2023)) | 
   ((df['games']>40) & (df['year']==2022)) | ((df['games_n1']>40) & (df['year_n1']==2022)) | 
   ((df['games']>33) & (df['year']==2021)) | ((df['games_n1']>33) & (df['year_n1']==2021)) | 
   ((df['games']>35) & (df['year']==2020)) | ((df['games_n1']>35) & (df['year_n1']==2020)) | 
   ((df['games']>40) & (df['year']==2019)) | ((df['games_n1']>40) & (df['year_n1']==2019)) |
   (df['year']==0)]['Player.Name'].index)
# Add Year in School KP
year_cond = [(df['Yr_n1']=='Fr'), (df['Yr_n1']=='So'), 
             (df['Yr_n1']=='Jr'), (df['Yr_n1']=='Sr')]
year_val = [1, 2, 3, 4]
df['Class_n1'] = np.select(year_cond, year_val)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['X.Min']*(df['ws']-min(df['ws']))
df['Calc A2'] = df['X.Min']*df['bpm']
# df['Calc A2'] = df['bpm']/(1-df['X.Min'])
df['Calc A3'] = df['X.Min']*df['ORtg']
df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
# df['Calc A2_n1'] = df['bpm_n1']/(1-df['X.Min_n1'])
df['Calc A_n1'] =df['X.Min_n1']*(df['ws_n1']-min(df['ws_n1']))
df['Calc A3_n1'] = df['X.Min_n1']*df['ORtg_n1']
df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR.'])*df['X.Min']
df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['X.Min']
df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
df['Calc Overall A'] = df['Calc A'] + df['Calc B']
df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
df['Calc Overall A_n1'] = df['Calc A_n1'] + df['Calc B_n1']
df['Calc Overall A2_n1'] = df['Calc A2_n1'] + df['Calc B_n1']
df['Calc Overall A3_n1'] = df['Calc A3_n1'] + df['Calc B_n1']
df[['Player.Name', 'player_type', 'season_n1', 'Team_n1', 'conf_abbr_n1', 'Year_dif', 'Team_AdjEM_dif', 'Class_n1',
    'KP_Conf_Dif', 'Calc A_n1', 'Calc A2_n1', 'Calc A3_n1', 'Calc B_n1', 'Calc C_n1', 'Calc Overall A_n1', 'Calc Overall A2_n1', 'Calc Overall A3_n1']]

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_11656/631237229.py:6: DtypeWarning: Columns (13,16,38,144,147,169) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv')


,Player.Name,player_type,season_n1,Team_n1,conf_abbr_n1,Year_dif,Team_AdjEM_dif,Class_n1,KP_Conf_Dif,Calc A_n1,Calc A2_n1,Calc A3_n1,Calc B_n1,Calc C_n1,Calc Overall A_n1,Calc Overall A2_n1,Calc Overall A3_n1
0,aamahne-santos-1,1,2019-20,JACKSONVILLE,A-Sun,1.0,0.000000,4,0.00,170.00,-258.40,6215.20,618.80,-516.80,788.80,360.40,6834.00
1,aamir-simms-1,1,2019-20,CLEMSON,ACC,1.0,0.000000,3,0.00,385.05,596.45,8093.60,1676.10,1155.15,2061.15,2272.55,9769.70
2,aamir-simms-1,1,2020-21,CLEMSON,ACC,1.0,0.000000,4,0.00,339.48,642.06,8324.64,1453.86,1461.24,1793.34,2095.92,9778.50
3,aanen-moody-1,1,2019-20,NORTH DAKOTA,Summit,1.0,0.000000,2,0.00,11.05,-24.65,742.90,112.20,-17.85,123.25,87.55,855.10
4,aanen-moody-1,0,2020-21,SOUTHERN UTAH,Big Sky,1.0,6.190432,3,1.27,52.63,5.54,3360.01,337.94,390.57,390.57,343.48,3697.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,zoar-nedd-1,1,2021-22,HAWAII,Big West,2.0,0.000000,2,0.00,15.60,-50.40,1006.80,195.60,-63.60,211.20,145.20,1202.40
11036,zoar-nedd-1,1,2022-23,HAWAII,Big West,1.0,0.000000,3,0.00,3.20,-21.12,273.92,25.60,-61.76,28.80,4.48,299.52
11039,zyon-pullin-1,1,2020-21,UC-RIVERSIDE,Big West,1.0,0.000000,2,0.00,316.52,285.64,8677.28,1358.72,1351.00,1675.24,1644.36,10036.00
11040,zyon-pullin-1,1,2021-22,UC-RIVERSIDE,Big West,1.0,0.000000,3,0.00,418.50,209.25,9006.12,1581.93,1205.28,2000.43,1791.18,10588.05


In [50]:
br_pos = pd.read_excel("/Users/caldavis/Documents/GitHub/Capstone-project/Player ID MASTER (2018-19 - 2022-23).xlsx")
br_pos = br_pos[['PLAYER_ID', 'Season', 'Pos']]
br_pos

,PLAYER_ID,Season,Pos
0,chris-clemons-2,2018-19,G
1,justin-wright-foreman-1,2018-19,G
2,antoine-davis-2,2022-23,G
3,carsen-edwards-1,2018-19,G
4,rj-barrett-1,2018-19,F
...,...,...,...
24794,eric-zalewski-1,2019-20,G
24795,eric-zalewski-1,2020-21,G
24796,ryan-zambie-1,2020-21,G
24797,ryan-zambie-1,2021-22,G


In [55]:
d = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")
d = d[['Player Name', 'season', 'Team', 'conf_abbr', 'games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots']]
d[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots']].apply(pd.to_numeric, errors="ignore")
d


/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_11656/51446953.py:1: DtypeWarning: Columns (10,13,16,19,35) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")


,Player Name,season,Team,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,G,S,%Min,ORtg,%Poss,%Shots
0,chris-clemons-2,2018-19,CAMPBELL,Big South,33,33,36.6,9.2,20.6,0.448,5.0,8.8,0.569,4.2,11.8,0.357,7.5,8.6,0.869,0.8,4.3,5.1,2.8,1.5,0.3,2.5,1.9,30.1,-4.39,1208,304.0,679.0,165.0,290.0,0.569,139.0,389.0,246.0,283.0,25.0,142.0,167,94.0,49.0,11.0,84.0,63.0,993.0,10.1,22.5,5.5,9.6,4.6,12.9,8.1,9.4,5.5,3.1,1.6,0.4,2.8,2.1,32.9,14.8,33.1,8.0,14.1,6.8,18.9,12.0,13.8,8.1,4.6,2.4,0.5,4.1,3.1,48.3,120.1,104.4,33.0,0.610,0.550,0.573,0.417,888.0,2.4,14.5,8.3,19.3,2.4,1.1,9.4,39.0,6.1,1.4,7.5,0.250,9.3,-1.6,7.7,2019,69,180.0,30,30.0,93.9,115.6,37.5,39.4
1,antoine-davis-2,2018-19,DETROIT MERCY,Horizon,30,29,37.4,8.8,21.9,0.4,4.4,10.3,0.423,4.4,11.6,0.38,4.2,4.9,0.857,0.7,2.4,3.1,3.6,0.9,0.0,3.3,2.3,26.1,-0.36,1122,263.0,657.0,131.0,310.0,0.423,132.0,347.0,126.0,147.0,22.0,72.0,94,107.0,28.0,0.0,99.0,70.0,784.0,9.4,23.4,4.7,11.1,4.7,12.4,4.5,5.2,3.4,3.8,1.0,0.0,3.5,2.5,28.0,13.3,33.3,6.6,15.7,6.7,17.6,6.4,7.4,4.8,5.4,1.4,0.0,5.0,3.5,39.7,109.0,118.9,22.4,0.539,0.501,0.528,0.224,738.0,2.1,7.5,4.7,24.0,1.4,0.0,12.0,36.4,3.4,-0.2,3.1,0.112,5.8,-3.6,2.2,2019,73,170.0,30,30.0,90.4,107.7,34.8,38.8
2,antoine-davis-2,2019-20,DETROIT MERCY,Horizon,30,30,36.7,7.9,20.8,0.38,4.5,10.4,0.437,3.4,10.4,0.324,5.1,5.7,0.901,0.3,2.8,3.1,4.5,1.7,0.1,4.5,2.5,24.3,-1.90,1102,237.0,623.0,136.0,311.0,0.437,101.0,312.0,154.0,171.0,9.0,85.0,94,134.0,52.0,3.0,136.0,75.0,729.0,8.6,22.6,4.9,11.3,3.7,11.3,5.6,6.2,3.4,4.9,1.9,0.1,4.9,2.7,26.5,12.2,32.0,7.0,16.0,5.2,16.0,7.9,8.8,4.8,6.9,2.7,0.2,7.0,3.9,37.5,101.8,112.5,21.0,0.518,0.461,0.501,0.274,702.0,0.9,8.8,4.7,33.2,2.7,0.3,16.2,37.2,2.5,0.2,2.7,0.097,3.4,-2.4,1.1,2020,73,160.0,30,30.0,88.6,100.0,36.1,38.0
3,antoine-davis-2,2020-21,DETROIT MERCY,Horizon,22,22,38.5,8.1,19.1,0.424,4.3,9.0,0.482,3.8,10.1,0.372,4.0,4.4,0.917,0.2,2.7,2.9,4.8,1.5,0.0,3.3,1.6,24.0,-6.21,846,178.0,420.0,95.0,197.0,0.482,83.0,223.0,88.0,96.0,4.0,60.0,64,105.0,32.0,0.0,72.0,36.0,527.0,8.4,19.9,4.5,9.3,3.9,10.5,4.2,4.5,3.0,5.0,1.5,0.0,3.4,1.7,24.9,12.3,29.1,6.6,13.7,5.8,15.5,6.1,6.7,4.4,7.3,2.2,0.0,5.0,2.5,36.5,112.0,111.7,23.5,0.566,0.523,0.531,0.229,509.0,0.6,8.6,4.6,26.7,2.2,0.0,13.4,33.1,3.0,0.3,3.3,0.155,3.9,-3.1,0.9,2021,73,165.0,22,22.0,95.6,110.5,32.0,34.1
4,carsen-edwards-1,2018-19,PURDUE,Big Ten,36,36,35.4,7.7,19.5,0.394,3.9,9.0,0.44,3.8,10.6,0.355,5.1,6.1,0.837,0.4,3.2,3.6,2.9,1.3,0.3,3.1,2.0,24.3,11.99,1275,277.0,703.0,142.0,323.0,0.44,135.0,380.0,185.0,221.0,14.0,116.0,130,104.0,48.0,10.0,113.0,73.0,874.0,8.7,22.1,4.5,10.1,4.2,11.9,5.8,6.9,4.1,3.3,1.5,0.3,3.5,2.3,27.4,13.1,33.4,6.7,15.3,6.4,18.0,8.8,10.5,6.2,4.9,2.3,0.5,5.4,3.5,41.5,109.9,103.4,23.2,0.541,0.490,0.541,0.314,787.0,1.3,10.6,6.0,18.3,2.3,1.0,12.3,37.3,4.0,1.6,5.6,0.176,7.5,1.0,8.5,2019,73,200.0,36,36.0,87.3,108.0,34.7,37.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
df = pd.merge(df, br_pos, left_on=['Player Name', 'season'], right_on=['PLAYER_ID', 'Season'], how='left')
#d = pd.merge(df, d, left_on=['season_n1', 'Team_n1', 'conf_abbr_n1'], right_on=[''])

In [40]:
kp = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KenPom Team Metrics.csv")
#kp.head(10)
kp_conf_rat = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KP_Conf_Ratings.csv")
kp_conf_rat = kp_conf_rat[["Year_Conf", "Conf_Abb", "Conf_Rk", "Conf_Full", "Conf_Rating"]]

d = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")
d = pd.merge(d, d2, left_on=['Player Name', 'season', 'Team', 'conf_abbr'],
             right_on=['Player.Name', 'season', 'Team', 'conf_abbr'], how='inner')
pd.set_option('display.max_columns', None)
d



/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_11656/1709202471.py:6: DtypeWarning: Columns (10,13,16,19,35) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")


,Unnamed: 0_x,Player Name,season,Team,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg2_per_g,fg2a_per_g,fg2_pct,fg3_per_g,fg3a_per_g,fg3_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,sos,mp,fg,fga,fg2,fg2a,fg2_pct.1,fg3,fg3a,ft,fta,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg_per_min,fga_per_min,fg2_per_min,fg2a_per_min,fg3_per_min,fg3a_per_min,ft_per_min,fta_per_min,trb_per_min,ast_per_min,stl_per_min,blk_per_min,tov_per_min,pf_per_min,pts_per_min,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,fg3_per_poss,fg3a_per_poss,ft_per_poss,fta_per_poss,trb_per_poss,ast_per_poss,stl_per_poss,blk_per_poss,tov_per_poss,pf_per_poss,pts_per_poss,off_rtg,def_rtg,per,ts_pct,efg_pct,fg3a_per_fga_pct,fta_per_fga_pct,pprod,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_40,obpm,dbpm,bpm,year,Ht,Wt,Yr,G,S,%Min,ORtg,%Poss,%Shots,eFG%,TS%,OR%,DR%,ARate,TORate,Blk%,Stl%,FC/40,FD/40,FTRate,FTM-A,Pct,2PM-A,Pct.1,3PM-A,Pct.2,%Pct,%2PM-A,%Pct.1,Player.Name,player_type,season_n1,Team_n1,conf_abbr_n1
0,8.0,antoine-davis-2,2018-19,DETROIT MERCY,Horizon,30,29,37.4,8.8,21.9,0.4,4.4,10.3,0.423,4.4,11.6,0.38,4.2,4.9,0.857,0.7,2.4,3.1,3.6,0.9,0.0,3.3,2.3,26.1,-0.36,1122,263.0,657.0,131.0,310.0,0.423,132.0,347.0,126.0,147.0,22.0,72.0,94,107.0,28.0,0.0,99.0,70.0,784.0,9.4,23.4,4.7,11.1,4.7,12.4,4.5,5.2,3.4,3.8,1.0,0.0,3.5,2.5,28.0,13.3,33.3,6.6,15.7,6.7,17.6,6.4,7.4,4.8,5.4,1.4,0.0,5.0,3.5,39.7,109.0,118.9,22.4,0.539,0.501,0.528,0.224,738.0,2.1,7.5,4.7,24.0,1.4,0.0,12.0,36.4,3.4,-0.2,3.1,0.112,5.8,-3.6,2.2,2019,73,170.0,Fr,30,30.0,90.4,107.7,34.8,38.8,50.1,53.9,2.1,7.5,24.0,14.5,0.0,1.4,2.5,5.0,22.4,126-147,.857 69,131-310,0.423,132-347,0.380,0.857,131-310,0.423,antoine-davis-2,1,2019-20,DETROIT MERCY,Horizon
1,9.0,antoine-davis-2,2019-20,DETROIT MERCY,Horizon,30,30,36.7,7.9,20.8,0.38,4.5,10.4,0.437,3.4,10.4,0.324,5.1,5.7,0.901,0.3,2.8,3.1,4.5,1.7,0.1,4.5,2.5,24.3,-1.90,1102,237.0,623.0,136.0,311.0,0.437,101.0,312.0,154.0,171.0,9.0,85.0,94,134.0,52.0,3.0,136.0,75.0,729.0,8.6,22.6,4.9,11.3,3.7,11.3,5.6,6.2,3.4,4.9,1.9,0.1,4.9,2.7,26.5,12.2,32.0,7.0,16.0,5.2,16.0,7.9,8.8,4.8,6.9,2.7,0.2,7.0,3.9,37.5,101.8,112.5,21.0,0.518,0.461,0.501,0.274,702.0,0.9,8.8,4.7,33.2,2.7,0.3,16.2,37.2,2.5,0.2,2.7,0.097,3.4,-2.4,1.1,2020,73,160.0,So,30,30.0,88.6,100.0,36.1,38.0,46.1,51.8,0.9,8.8,33.1,19.4,0.3,2.7,2.7,5.6,27.4,154-171,.901 16,136-311,0.437,101-312,0.324,0.901,136-311,0.437,antoine-davis-2,1,2020-21,DETROIT MERCY,Horizon
2,10.0,antoine-davis-2,2020-21,DETROIT MERCY,Horizon,22,22,38.5,8.1,19.1,0.424,4.3,9.0,0.482,3.8,10.1,0.372,4.0,4.4,0.917,0.2,2.7,2.9,4.8,1.5,0.0,3.3,1.6,24.0,-6.21,846,178.0,420.0,95.0,197.0,0.482,83.0,223.0,88.0,96.0,4.0,60.0,64,105.0,32.0,0.0,72.0,36.0,527.0,8.4,19.9,4.5,9.3,3.9,10.5,4.2,4.5,3.0,5.0,1.5,0.0,3.4,1.7,24.9,12.3,29.1,6.6,13.7,5.8,15.5,6.1,6.7,4.4,7.3,2.2,0.0,5.0,2.5,36.5,112.0,111.7,23.5,0.566,0.523,0.531,0.229,509.0,0.6,8.6,4.6,26.7,2.2,0.0,13.4,33.1,3.0,0.3,3.3,0.155,3.9,-3.1,0.9,2021,73,165.0,Jr,22,22.0,95.6,110.5,32.0,34.1,52.3,56.6,0.6,8.5,26.7,15.6,0.0,2.2,1.7,4.7,22.9,88-96,.917 7,95-197,0.482,83-223,0.372,0.917,95-197,0.482,antoine-davis-2,1,2021-22,DETROIT MERCY,Horizon
3,18.0,jhivvan-jackson-1,2018-19,UTSA,CUSA,29,24,30.8,7.7,19.8,0.386,4.1,9.6,0.423,3.6,10.2,0.351,4.0,4.7,0.846,0.4,3.6,4.1,2.4,1.2,0.1,2.3,2.8,22.9,0.05,894,222.0,575.0,118.0,279.0,0.423,104.0,296.0,115.0,136.0,13.0,105.0,118,71.0,35.0,2.0,68.0,80.0,663.0,9.9,25.7,5.3,12.5,4.7,13.2,5.1,6.1,5.3,3.2,1.6,0.1,3.0,3.6,29.7,13.4,34.8,7.1,16.9,6.3,17.9,7.0,8.2,7.1,4.3,2.1,0.1,4.1,4.8,40.1,104.1,102.5,22.1,0.518,0.477,0.515,0.237,599.0,1.5,12.5,6.9,17.9,2.1,0.2,9.6,37.6,2.0,1.2,3.2,0.143,4.3,-1.1,3.1,2019,72,170.0,So,26,21.0,71.1,100.7,35.4,40.1,46.5,50.7,1.5,11.4,18.1,11.7,0.3,2.1,3.7,5.5,24.2,103-124,.831 152,102-250,0.408,91-263,0.346,0.831,102-250,0.408,jhivvan-jackson-1,1,2019-20,UTSA,CUSA
4,19.0,jhivvan-jackson-1,2019-20,UTSA,CUSA,32,31,34.5,

In [39]:
df = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
# d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
# df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, br_pos, left_on=['Player Name', 'season'], right_on=['PLAYER_ID', 'Season'], how='left')
d#f = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = df.replace(to_replace="NO IDEA", value=0)
df = df.replace(to_replace="---", value=0)
df = df.fillna(0)

# Convert to Numeric
df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots']].apply(pd.to_numeric, errors="ignore")
    #'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']] = df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']].apply(pd.to_numeric, errors="ignore")

# Add Contributions
# conditions = [(df['X.Poss_n1']>=28.0), (df['X.Poss_n1']>=24.0), (df['X.Poss_n1']>=20.0), (df['X.Poss_n1']>=16), 
#               (df['X.Poss_n1']>=12.0), (df['X.Poss_n1']<12)]
#                                         #& (df['X.Min_n1']>=10.0))]
# #(df['X.Min_n1']<10),
# #values = ['Go-To Guy', 'Major Contributor', 'Significant Contributor', 'Role Players', 'Limited Role', 'Nearly Invisible', 'Bench Warmer']
# values = [0, 1, 2, 3, 4, 5]
# df['Contribution_n1'] = np.select(conditions, values)

# Add Year in School KP
year_cond = [(df['Yr_n1']=='Fr'), (df['Yr_n1']=='So'), 
             (df['Yr_n1']=='Jr'), (df['Yr_n1']=='Sr')]
year_val = [1, 2, 3, 4]
df['Class_n1'] = np.select(year_cond, year_val)

# year_cond = [(df['Yr']=='Fr'), (df['Yr']=='So'), 
#              (df['Yr']=='Jr'), (df['Yr']=='Sr')]
# year_val = [1, 2, 3, 4]
# df['Class'] = np.select(year_cond, year_val)

# # Add Ht. Categories
# ht_cond = [(df['Ht']<=74), (df['Ht']>74) & (df['Ht']<=76), (df['Ht']>76) & (df['Ht']<=78),
#            (df['Ht']>78) & (df['Ht']<=80), (df['Ht']>80)]

# ht_val = [0, 1, 2, 3, 4]
# df['Pos_Ht'] = np.select(ht_cond, ht_val)
#df[‘column’] =( df[‘column’] – df[‘column’].mean() ) / df[‘column’].std()


# Additional Features + Model Prep
#df = df.drop(df[(df['games']<=10) & (df['games_started']/df['games']>0.25)].index)
#df = df.drop(df[(df['games']<=3)].index)
df = df.drop(df.loc[((df['games']>39) & (df['year']==2023)) | ((df['games']>40) & (df['year']==2022)) |
                    ((df['games']>33) & (df['year']==2021)) | ((df['games']>35) & (df['year']==2020)) | 
                    ((df['games']>40) & (df['year']==2019)) | (df['year']==0)]['Player Name'].index)
                    #| ((df['games_n1']>39) & (df['year_n1']==2023)) | 
    # ((df['games_n1']>40) & (df['year_n1']==2022)) | 
    # ((df['games_n1']>33) & (df['year_n1']==2021)) | 
    # | ((df['games_n1']>35) & (df['year_n1']==2020)) | 
    # ((df['games_n1']>40) & (df['year_n1']==2019)) |
#df = df.drop(df[(df['%Min']<=10)].index)
#df['Year_dif'] = df['year_n1']-df['year']
#df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
#df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
#df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['%Min']*df['ORtg']
df['Calc A2'] = df['%Min']*df['bpm']
df['Calc A3'] = df['%Min']*df['ORtg']
#df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
#df['Calc A_n1'] =df['X.Min_n1']*df['ORtg_n1']
df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR%'])*df['%Min']
#df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['%Min']
#df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
#df['Calc Overall'] = df['Calc A'] + df['Calc B']
df['Calc Overall A'] = df['Calc A'] + df['Calc B']
df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
#df['Calc Overall_n1'] = df['Calc A_n1'] + df['Calc B_n1']
df = df.loc[df['year'].isin([2022, 2023])]


df

KeyError: 'year_n1'

In [4]:
df['fg_pct']=df['fg_pct'].astype(str).astype(float)
df['fg2_pct']=df['fg2_pct'].astype(str).astype(float)
df['fg3_pct']=df['fg3_pct'].astype(str).astype(float)
df['ft_pct']=df['ft_pct'].astype(str).astype(float)

In [5]:
df[df['Pos']==0]

,Unnamed: 0_x,Player Name,season,Team,conf_abbr,games,games_started,mp_per_g,fg_per_g,fga_per_g,...,Conf_Full,Conf_Rating,PLAYER_ID,Season_y,Pos,Calc A,Calc A2,Calc B,Calc C,Calc Overall
14924,28013.0,tre-thomas-2,2022-23,HAMPTON,CAA,13,2,10.2,1.5,4.0,...,Colonial Athletic Association,-7.58,tre-thomas-2,2022-23,0,758.40,-81.60,113.28,-319.68,871.68
16742,32111.0,cory-barnes-1,2022-23,DELAWARE STATE,MEAC,20,0,4.9,0.2,0.7,...,Mid-Eastern Athletic Conference,-9.72,cory-barnes-1,2022-23,0,438.82,-92.50,132.46,-371.48,571.28
19164,30259.0,machai-nelson-1,2022-23,HARTFORD,Ind,15,0,3.9,0.5,0.9,...,Independents,-19.79,machai-nelson-1,2022-23,0,419.02,-25.01,29.11,7.79,448.13


In [6]:
df_g = df[df['Pos']=='G']
print(len(df_g))

df_f = df[df['Pos']=='F']
print(len(df_f))

df_c = df[df['Pos']=='C']
print(len(df_c))

4747
2911
404


In [23]:
h = ['player_type', 'Calc A2', 'ORtg', 'KP_Conf_Dif', 'orb_pct', 'Calc A3', 'Calc A2', 'Calc B', 'bpm', 'tov_pct', 'efg_pct', 'ws', 'Year_dif']
print(len(h))
h = list(set(h))


13


TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [24]:
# df_simp = df[['fg_pct', 'fg2_pct', 'fg3_pct', 'ft_pct', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
#     'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 
#     'Ht', 'Wt', 'Conf_Rating', 'sos']]

df_simp = df[['player_type', 'Calc A2', 'ORtg', 'KP_Conf_Dif', 'orb_pct', 'Calc A3', 'Calc A2', 'Calc B', 'bpm', 'tov_pct', 'efg_pct', 'ws', 'Year_dif']]
df_simp = list(set(df_simp))

df_std = (df_simp-df_simp.mean())/df_simp.std()
df_name = df[['Player Name', 'year', 'Pos']]
df_name['Player_Year'] = df_name['Player Name'] + "_" + df_name['year'].astype(int).astype(str)
df_std = pd.merge(df_name, df_std, left_index=True, right_index=True)
df_std = df_std.drop(['Player Name', 'year'], axis=1)

KeyError: "['player_type', 'KP_Conf_Dif', 'Calc A3', 'Year_dif'] not in index"

In [8]:
df_std

,Pos,Player_Year,fg_pct,fg2_pct,fg3_pct,ft_pct,fg_per_poss,fga_per_poss,fg2_per_poss,fg2a_per_poss,...,dws,ws,ws_per_40,obpm,dbpm,bpm,Ht,Wt,Conf_Rating,sos
15,F,keegan-murray-1_2022,0.661421,0.661768,0.653067,0.617614,3.405654,2.519965,2.717946,2.161524,...,2.260787,5.155486,0.232796,2.618647,0.597582,3.597149,0.771928,0.672208,1.564681,1.305202
19,G,darius-mcghee-1_2022,0.197332,0.410800,0.619532,0.990284,3.212798,3.475599,0.887247,0.724311,...,2.260787,3.661415,0.139893,1.601363,-0.133914,1.794446,-2.509727,-5.221769,-0.666901,-0.725050
20,G,darius-mcghee-1_2023,0.116827,0.294969,0.636300,0.951348,2.981371,3.495508,0.374652,0.338194,...,3.293062,4.230585,0.165002,1.952788,0.338019,2.600361,-2.509727,-0.506587,-0.369742,-0.333182
26,G,daylen-kountz-1_2022,0.462526,0.356745,0.749480,0.828979,2.325662,1.882876,1.912439,1.925564,...,0.024191,2.665368,0.048245,0.935504,-0.558654,0.649199,-0.421401,-0.244633,-0.838028,-0.661605
27,G,daylen-kountz-1_2023,0.112091,0.082611,0.368022,0.742765,1.207098,1.424968,1.216773,1.646701,...,-0.836038,0.602127,-0.050935,0.343630,-0.605848,-0.093090,-0.421401,-0.244633,-0.302679,-0.098060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20600,G,mannie-harris-1_2023,2.773502,2.125103,-1.015287,1.321237,-0.490033,0.369790,-1.199749,-1.163373,...,-0.836038,-0.891944,-0.093620,-0.784630,-0.369881,-1.153504,-0.719734,-0.506587,0.012980,-0.353708
20606,G,alsean-evans-1_2022,-0.385148,-0.145190,-1.015287,1.321237,-0.065750,0.867515,0.630950,1.367839,...,-1.008084,-1.176528,-0.315834,-1.709434,-0.369881,-2.235126,0.175263,-0.113655,-0.265679,-0.112989
20618,F,shanquan-hemphill-1_2022,0.614065,0.360607,1.780674,0.295005,0.859958,0.210517,1.473071,1.432192,...,1.056466,1.242444,0.035691,0.584079,0.243632,0.946115,0.175263,0.017322,0.002573,0.217300
20620,G,charles-obannonjr-1_2022,-0.029977,0.132805,0.401556,0.495245,0.358532,0.668425,-0.174558,-0.155178,...,1.400558,1.313590,0.045734,1.027985,0.739162,1.900487,0.175263,0.410254,2.169406,1.919129


In [9]:
#df_std = df_std.drop(['Pos'], axis=1)

In [10]:
df_g = df_std[df_std['Pos']=='G']
df_g = df_g.drop(['Pos'], axis=1)

df_f = df_std[df_std['Pos']=='F']
df_f = df_f.drop(['Pos'], axis=1)

df_c = df_std[df_std['Pos']=='C']
df_c = df_c.drop(['Pos'], axis=1)

In [11]:
def magnitude(v, cols):
  word = (math.sqrt(sum(v[n].iloc[0]*v[n].iloc[0] for n in cols)))
  return word

def cosine(v1, v2, cols):
  dot_product = sum([v1[i].iloc[0]*v2[i].iloc[0] for i in cols])
  return(dot_product/(magnitude(v1, cols)*magnitude(v2, cols)))

#print(cosine(c, d, c.columns))
#matrix = pd.DataFrame()


In [12]:
df_std = df_std.drop(['Pos'], axis=1)

In [13]:

a_list = ['zach-edey-1_2022', 'zach-edey-1_2023', 'hunter-dickinson-1_2022', 'hunter-dickinson-1_2023', 
          'jake-stephens-1_2022', 'tre-mitchell-1_2022', 'tre-mitchell-1_2023']

b_list = list(df_std['Player_Year'].unique())
#print(type(a_list))
matrix = {'id1': [], 'id2': [], 'Similarity': []}
for id1 in a_list:
  print(id1)
  #print(id1, type(id1), )
  for id2 in b_list:
    if id1 == id2: continue
    v1 = df_std[df_std['Player_Year']==id1]
    v1 = v1.drop('Player_Year', axis=1)
    v2 = df_std[df_std['Player_Year']==id2]
    v2 = v2.drop('Player_Year', axis=1)
    matrix['id1'].append(id1)
    matrix['id2'].append(id2)
    #print(v1.columns)
    matrix['Similarity'].append(cosine(v1,v2, list(v1.columns)))
    #v1.columns
    #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

sim_matrix = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

sim_matrix


#matrix

zach-edey-1_2022
zach-edey-1_2023
hunter-dickinson-1_2022
hunter-dickinson-1_2023
jake-stephens-1_2022
tre-mitchell-1_2022
tre-mitchell-1_2023


,id1,id2,Similarity
0,zach-edey-1_2022,keegan-murray-1_2022,0.824580
1,zach-edey-1_2022,darius-mcghee-1_2022,0.364520
2,zach-edey-1_2022,darius-mcghee-1_2023,0.380475
3,zach-edey-1_2022,daylen-kountz-1_2022,0.591199
4,zach-edey-1_2022,daylen-kountz-1_2023,0.413871
...,...,...,...
56394,tre-mitchell-1_2023,jerry-higginsiii-1_2022,-0.255071
56395,tre-mitchell-1_2023,devin-carter-2_2022,-0.421098
56396,tre-mitchell-1_2023,devin-carter-2_2023,-0.725137
56397,tre-mitchell-1_2023,mannie-harris-1_2023,-0.432821


In [14]:
sim_matrix[sim_matrix['id1']=='zach-edey-1_2022'].sort_values('Similarity', ascending=False).head(5)

,id1,id2,Similarity
176,zach-edey-1_2022,kofi-cockburn-1_2022,0.962017
7,zach-edey-1_2022,zach-edey-1_2023,0.957756
770,zach-edey-1_2022,colin-castleton-1_2022,0.951927
6692,zach-edey-1_2022,tolu-smith-1_2023,0.940547
6797,zach-edey-1_2022,nfaly-dante-1_2023,0.937940


In [15]:
from scipy import spatial
# a_list = list(df_c['Player_Year'].unique())
# b_list = a_list
a_list = ['zach-edey-1_2022', 'zach-edey-1_2023', 'hunter-dickinson-1_2022', 'hunter-dickinson-1_2023']
 #         'jake-stephens-1_2022', 'tre-mitchell-1_2022', 'tre-mitchell-1_2023']

b_list = list(df_c['Player_Year'].unique())
#print(type(a_list))
matrix = {'id1': [], 'id2': [], 'Similarity': []}
for id1 in a_list:
  print(id1)
  #print(id1, type(id1), )
  for id2 in b_list:
    if id1 == id2: continue
    v1 = df_c[df_c['Player_Year']==id1]
    v1 = v1.drop('Player_Year', axis=1)
    v2 = df_c[df_c['Player_Year']==id2]
    v2 = v2.drop('Player_Year', axis=1)
    matrix['id1'].append(id1)
    matrix['id2'].append(id2)
    #matrix['Similarity'].append(1-spatial.distance.cosine(v1.iloc[0], v2.iloc[0]))
    #v1.columns
    #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

sim_matrix_c = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

sim_matrix_c

zach-edey-1_2022
zach-edey-1_2023
hunter-dickinson-1_2022
hunter-dickinson-1_2023


ValueError: All arrays must be of the same length

In [17]:
#df_c = df_std[df_std['Pos']=='C']
#df_c = df_c.drop(['Pos'], axis=1)
df_c_col = list(df_c['Player_Year'])
df_c = df_c.set_index('Player_Year')

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
a = cosine_similarity(df_c, df_c)
df2 = pd.DataFrame(a, columns=df_c_col)
df2['Temp'] = df_c_col
df2 = df2.set_index('Temp')

In [19]:
df2

,zach-edey-1_2022,zach-edey-1_2023,hunter-dickinson-1_2022,hunter-dickinson-1_2023,jake-stephens-1_2022,jake-stephens-1_2023,kofi-cockburn-1_2022,ahsan-asadullah-1_2022,ahsan-asadullah-1_2023,morris-udeze-1_2022,...,yusuf-jihad-1_2022,yusuf-jihad-1_2023,joe-hedstrom-1_2022,joe-hedstrom-1_2023,zeke-richards-1_2022,zeke-richards-1_2023,grant-asman-1_2022,grant-asman-1_2023,ludgy-debaut-1_2022,ludgy-debaut-1_2023
Temp,,,,,,,,,,,,,,,,,,,,,
zach-edey-1_2022,1.000000,0.957756,0.899990,0.886203,0.794896,0.864522,0.962017,0.766709,0.383357,0.825052,...,0.069285,-0.355053,-0.059171,0.264781,0.368341,0.123550,0.104875,0.220446,0.052619,0.118861
zach-edey-1_2023,0.957756,1.000000,0.924135,0.939474,0.837739,0.862509,0.946686,0.700554,0.336843,0.767564,...,-0.002862,-0.414245,-0.160509,0.193763,0.333606,0.015092,0.056397,0.202789,0.059795,0.076729
hunter-dickinson-1_2022,0.899990,0.924135,1.000000,0.966640,0.869014,0.858247,0.887623,0.717715,0.298416,0.693131,...,-0.124412,-0.311001,-0.098703,0.175420,0.228096,-0.017442,-0.018206,0.249488,0.051091,-0.054319
hunter-dickinson-1_2023,0.886203,0.939474,0.966640,1.000000,0.852399,0.843356,0.894001,0.658615,0.283669,0.703266,...,-0.128733,-0.363222,-0.118904,0.129837,0.222810,-0.082503,-0.008826,0.272840,0.062861,-0.028166
jake-stephens-1_2022,0.794896,0.837739,0.869014,0.852399,1.000000,0.959781,0.742450,0.666091,0.214858,0.473271,...,-0.258231,-0.364159,-0.234403,0.246128,0.199320,-0.126838,0.075670,0.348952,-0.055413,-0.165039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zeke-richards-1_2023,0.123550,0.015092,-0.017442,-0.082503,-0.126838,-0.108709,0.074866,0.018004,0.224729,0.466756,...,0.457399,0.145847,0.299582,0.482796,0.678155,1.000000,-0.026613,-0.096379,0.417543,0.681204
grant-asman-1_2022,0.104875,0.056397,-0.018206,-0.008826,0.075670,0.236686,0.013929,0.297101,0.440567,-0.010113,...,-0.077800,-0.241226,0.285857,-0.060038,-0.036139,-0.026613,1.000000,0.426895,-0.069668,0.072926
grant-asman-1_2023,0.220446,0.202789,0.249488,0.272840,0.348952,0.385291,0.062333,0.296422,0.234905,0.049069,...,0.119146,-0.062878,0.057921,0.289399,0.091059,-0.096379,0.426895,1.000000,0.236773,0.081609


In [20]:
df2['zach-edey-1_2022'].sort_values(ascending=False).head()

Temp
zach-edey-1_2022        1.000000
kofi-cockburn-1_2022    0.962017
zach-edey-1_2023        0.957756
nfaly-dante-1_2023      0.937940
oumar-ballo-1_2023      0.908891
Name: zach-edey-1_2022, dtype: float64

In [ ]:
#sim_matrix_c.to_csv("Similarity_Matrix_Centers.csv")

In [ ]:
sim_matrix_c[sim_matrix_c['id1']=='zach-edey-1_2023'].sort_values('Similarity', ascending=False).head(5)

#sim_matrix_c[sim_matrix_c['id1']=='hunter-dickinson-1_2022'].sort_values('Similarity', ascending=False).head(5)

#sim_matrix_c[sim_matrix_c['id1']=='zeke-richards-1_2022'].sort_values('Similarity', ascending=False).head(5)

,id1,id2,Similarity
305,zach-edey-1_2023,zach-edey-1_2022,0.956708
310,zach-edey-1_2023,kofi-cockburn-1_2022,0.944014
307,zach-edey-1_2023,hunter-dickinson-1_2023,0.939045
593,zach-edey-1_2023,oumar-ballo-1_2023,0.935382
314,zach-edey-1_2023,morris-udeze-1_2023,0.928507


In [ ]:
from scipy import spatial
df_f = df_f.sort_values('Player_Year')
a_list = list(df_f['Player_Year'].unique())
b_list = a_list

#print(type(a_list))
matrix = {'id1': [], 'id2': [], 'Similarity': []}
for id1 in a_list:
  print(id1)
  #print(id1, type(id1), )
  for id2 in b_list:
    if id1 == id2: continue
    v1 = df_f[df_f['Player_Year']==id1]
    v1 = v1.drop('Player_Year', axis=1)
    v2 = df_f[df_f['Player_Year']==id2]
    v2 = v2.drop('Player_Year', axis=1)
    matrix['id1'].append(id1)
    matrix['id2'].append(id2)
    matrix['Similarity'].append(1-spatial.distance.cosine(v1.iloc[0], v2.iloc[0]))
    #v1.columns
    #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

sim_matrix_f = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

sim_matrix_f

aaron-murphy-1_2022
aaron-reddish-1_2022
aaron-reddish-1_2023
aaron-wheeler-2_2022
abayomi-iyiola-1_2022
abayomi-iyiola-1_2023
abdou-ndiaye-1_2022
abdou-samb-1_2023
abdou-tsimbila-1_2022
abdou-tsimbila-1_2023
abdul-ado-1_2022
abdul-momoh-1_2023
abe-johnson-1_2023
abou-ousmane-1_2022
abou-ousmane-1_2023
abu-kigab-1_2022
adam-larson-1_2023
adam-thistlewood-1_2022
adama-sanogo-1_2022
adama-sanogo-1_2023
adem-bona-1_2023
ademide-badmus-1_2023
aguek-arop-1_2022
aguek-arop-1_2023
ahjany-lee-1_2023
ahmard-harvey-1_2023
ahmir-langlais-1_2022
ahmir-langlais-1_2023
aidan-igiehon-1_2022
aidan-igiehon-1_2023
aidan-shaw-1_2023
airion-simmons-1_2022
airion-simmons-1_2023
aj-allenspach-1_2022
aj-casey-1_2023
aj-clayton-1_2022
aj-clayton-1_2023
aj-edu-1_2023
aj-george-1_2023
aj-griffin-1_2022
aj-taylor-1_2022
aj-wilson-2_2022
ajang-aguek-1_2022
ajiri-johnson-1_2022
ajiri-johnson-1_2023
akeem-odusipe-1_2022
akok-akok-1_2022
akok-akok-1_2023
akol-arop-1_2022
akol-arop-1_2023
alanzo-frink-1_2022
alden-ap

,id1,id2,Similarity
0,aaron-murphy-1_2022,aaron-reddish-1_2022,0.225010
1,aaron-murphy-1_2022,aaron-reddish-1_2023,0.006931
2,aaron-murphy-1_2022,aaron-wheeler-2_2022,-0.292554
3,aaron-murphy-1_2022,abayomi-iyiola-1_2022,0.485996
4,aaron-murphy-1_2022,abayomi-iyiola-1_2023,0.575343
...,...,...,...
5815327,zurabi-zhgenti-1_2022,zinou-bedri-1_2023,0.640234
5815328,zurabi-zhgenti-1_2022,zion-griffin-1_2022,-0.273897
5815329,zurabi-zhgenti-1_2022,zion-griffin-1_2023,0.059453
5815330,zurabi-zhgenti-1_2022,zion-tordoff-1_2022,0.449062


In [ ]:
sim_matrix_f.to_csv("Similarity_Matrix_Forwards.csv")

In [ ]:
#sim_matrix_f[sim_matrix_f['id1']=='drew-timme-1_2022'].sort_values('Similarity', ascending=False).head(5)

sim_matrix_f[sim_matrix_f['id1']=='keegan-murray-1_2022'].sort_values('Similarity', ascending=False).head(5)

sim_matrix_f[sim_matrix_f['id1']=='keve-aluma-1_2022'].sort_values('Similarity', ascending=False).head(5)

,id1,id2,Similarity
3339254,keve-aluma-1_2022,adama-sanogo-1_2023,0.980305
3339406,keve-aluma-1_2022,azuolas-tubelis-1_2023,0.972410
3341115,keve-aluma-1_2022,paolo-banchero-1_2022,0.961324
3339801,keve-aluma-1_2022,deandre-williams-1_2023,0.960356
3339890,keve-aluma-1_2022,drew-timme-1_2023,0.960145


In [ ]:
from scipy import spatial
df_g = df_g.sort_values('Player_Year')
a_list = list(df_g['Player_Year'].unique())
b_list = a_list

#print(type(a_list))
matrix = {'id1': [], 'id2': [], 'Similarity': []}
for id1 in a_list:
  print(id1)
  #print(id1, type(id1), )
  for id2 in b_list:
    if id1 == id2: continue
    v1 = df_g[df_g['Player_Year']==id1]
    v1 = v1.drop('Player_Year', axis=1)
    v2 = df_g[df_g['Player_Year']==id2]
    v2 = v2.drop('Player_Year', axis=1)
    matrix['id1'].append(id1)
    matrix['id2'].append(id2)
    matrix['Similarity'].append(1-spatial.distance.cosine(v1.iloc[0], v2.iloc[0]))
    #v1.columns
    #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

sim_matrix_g = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

sim_matrix_g

aamer-muhammad-1_2023
aanen-moody-1_2022
aanen-moody-1_2023
aaron-cash-1_2022
aaron-cash-1_2023
aaron-clarke-1_2022
aaron-cook-2_2022
aaron-cooley-1_2022
aaron-cooley-1_2023
aaron-deloney-1_2022
aaron-deloney-1_2023
aaron-duhart-1_2022
aaron-estrada-1_2022
aaron-estrada-1_2023
aaron-gray-2_2023
aaron-lemon-warren-1_2023
aaron-scott-5_2022
aaron-scott-5_2023
aaron-thompson-3_2022
aaryn-rai-1_2022
abdoulaye-thiam-1_2023
abdul-mohamed-1_2022
aboubacar-traore-1_2022
aboubacar-traore-1_2023
ac-curry-1_2023
achile-spadone-1_2023
adam-flagler-1_2022
adam-flagler-1_2023
adam-hess-2_2023
adam-kunkel-1_2022
adam-kunkel-1_2023
adam-miller-3_2023
adam-seiko-1_2022
adam-seiko-1_2023
adam-silas-1_2023
adama-bal-1_2023
adante-holiman-1_2023
ade-adebayo-1_2022
ade-adebayo-1_2023
adefolalrin-adetogun-1_2022
adefolalrin-adetogun-1_2023
adetokunbo-bakare-1_2023
adham-eleeda-1_2022
adonis-arms-1_2022
adou-thiero-1_2023
adrian-balwinjr-1_2022
adrian-balwinjr-1_2023
adrian-delph-1_2022
adrian-martin-2_2022


,id1,id2,Similarity
0,aamer-muhammad-1_2023,aanen-moody-1_2022,0.320461
1,aamer-muhammad-1_2023,aanen-moody-1_2023,0.518100
2,aamer-muhammad-1_2023,aaron-cash-1_2022,-0.698760
3,aamer-muhammad-1_2023,aaron-cash-1_2023,0.013353
4,aamer-muhammad-1_2023,aaron-clarke-1_2022,0.561260
...,...,...,...
16986757,zytarious-mortle-1_2023,zion-russell-1_2022,0.339291
16986758,zytarious-mortle-1_2023,zion-styles-1_2023,0.600846
16986759,zytarious-mortle-1_2023,zion-williams-2_2022,0.456867
16986760,zytarious-mortle-1_2023,zyon-pullin-1_2022,-0.074825


In [ ]:
sim_matrix_g.to_csv("Similarity_Matrix_Guards.csv")

In [ ]:
#sim_matrix['True'] = 1-sim_matrix['Similarity']
sim_matrix = pd.concat([sim_matrix_c, sim_matrix_f, sim_matrix_g])
sim_matrix

,id1,id2,Similarity
0,zach-edey-1_2022,zach-edey-1_2023,0.956708
1,zach-edey-1_2022,hunter-dickinson-1_2022,0.900140
2,zach-edey-1_2022,hunter-dickinson-1_2023,0.885337
3,zach-edey-1_2022,jake-stephens-1_2022,0.795056
4,zach-edey-1_2022,jake-stephens-1_2023,0.863639
...,...,...,...
16986757,zytarious-mortle-1_2023,zion-russell-1_2022,0.339291
16986758,zytarious-mortle-1_2023,zion-styles-1_2023,0.600846
16986759,zytarious-mortle-1_2023,zion-williams-2_2022,0.456867
16986760,zytarious-mortle-1_2023,zyon-pullin-1_2022,-0.074825


In [ ]:
sim_matrix.to_csv("Full_Similarity_Matrix.csv")

In [ ]:
sim_matrix[sim_matrix['id1']=='james-bishop-2_2020'].sort_values('Similarity', ascending=False).head(5)

sim_matrix[sim_matrix['id1']=='brendan-adams-1_2022'].sort_values('Similarity', ascending=False).head(5)

sim_matrix[sim_matrix['id1']=='keegan-murray-1_2022'].sort_values('Similarity', ascending=False).head(5)

,id1,id2,Similarity
151,keegan-murray-1_2022,david-roddy-1_2022,0.953149
211,keegan-murray-1_2022,keve-aluma-1_2022,0.951684
116,keegan-murray-1_2022,kj-williams-1_2023,0.949189
173,keegan-murray-1_2022,kris-murray-1_2023,0.948218
107,keegan-murray-1_2022,deandre-williams-1_2023,0.921589


In [ ]:
# from scipy import spatial
# a_list = list(df_std['Player_Year'].unique())
# a_list = a_list[4800:5300]
# #print(type(a_list))
# matrix = {'id1': [], 'id2': [], 'Similarity': []}
# for id1 in a_list:
#   print(id1)
#   #print(id1, type(id1), )
#   for id2 in a_list:
#     if id1 == id2: continue
#     v1 = df_std[df_std['Player_Year']==id1]
#     v1 = v1.drop('Player_Year', axis=1)
#     v2 = df_std[df_std['Player_Year']==id2]
#     v2 = v2.drop('Player_Year', axis=1)
#     matrix['id1'].append(id1)
#     matrix['id2'].append(id2)
#     matrix['Similarity'].append(1-spatial.distance.cosine(v1.iloc[0], v2.iloc[0]))
#     #v1.columns
#     #= matrix{'id1':id1, 'id2':id2, 'similarity':cosine(v1,v2, v1.columns)}]))

# sim_matrix2 = pd.DataFrame(matrix, columns=['id1', 'id2', 'Similarity'])

# sim_matrix2

In [ ]:
# a = list(df_std['Player_Year'].unique())
# a = a[0:4800]
# a[4799]

'kamani-johnson-1_2022'